In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup

In [ ]:
lit_period = 4.887803076
lit_t0 = 124.8130808

In [ ]:
import numpy as np
import lightkurve as lk
from collections import OrderedDict

kepler_lcfs = lk.search_lightcurvefile("HAT-P-11", mission="Kepler").download_all()
kepler_lc = kepler_lcfs.PDCSAP_FLUX.stitch().remove_nans()
kepler_t = np.ascontiguousarray(kepler_lc.time, dtype=np.float64)
kepler_y = np.ascontiguousarray(1e3 * (kepler_lc.flux - 1), dtype=np.float64)
kepler_yerr = np.ascontiguousarray(1e3 * kepler_lc.flux_err, dtype=np.float64)

hdr = kepler_lcfs[0].hdu[1].header
kepler_texp = hdr["FRAMETIM"] * hdr["NUM_FRM"]
kepler_texp /= 60.0 * 60.0 * 24.0

tess_lcfs = lk.search_lightcurvefile("HAT-P-11", mission="TESS").download_all()
tess_lc = tess_lcfs.PDCSAP_FLUX.stitch().remove_nans()
tess_t = np.ascontiguousarray(tess_lc.time + 2457000 - 2454833, dtype=np.float64)
tess_y = np.ascontiguousarray(1e3 * (tess_lc.flux - 1), dtype=np.float64)
tess_yerr = np.ascontiguousarray(1e3 * tess_lc.flux_err, dtype=np.float64)

hdr = tess_lcfs[0].hdu[1].header
tess_texp = hdr["FRAMETIM"] * hdr["NUM_FRM"]
tess_texp /= 60.0 * 60.0 * 24.0

datasets = OrderedDict(
    [
        ("Kepler", [kepler_t, kepler_y, kepler_yerr, kepler_texp]),
        ("TESS", [tess_t, tess_y, tess_yerr, tess_texp]),
    ]
)

for name, (t, y, _, _) in datasets.items():
    plt.plot(t, y, label=name)
plt.legend(fontsize=10)
plt.xlabel("time [KBJD]")
_ = plt.ylabel("relative flux [ppt]")

In [ ]:
import pymc3 as pm
import exoplanet as xo
import theano.tensor as tt
from functools import partial

x_min = min(np.min(x) for x, _, _, _ in datasets.values())
x_max = max(np.max(x) for x, _, _, _ in datasets.values())
x_mid = 0.5 * (x_min + x_max)
t0_ref = lit_t0 + lit_period * np.round((x_mid - lit_t0) / lit_period)

for i in range(10):
    with pm.Model() as model:

        # Shared orbital parameters
        period = pm.Lognormal("period", mu=np.log(lit_period), sigma=1.0)
        t0 = pm.Normal("t0", mu=t0_ref, sigma=1.0)
        dur = pm.Lognormal("dur", mu=np.log(0.1), sigma=10.0)
        b = xo.UnitUniform("b")
        ld_arg = 1 - tt.sqrt(1 - b ** 2)
        orbit = xo.orbits.KeplerianOrbit(period=period, duration=dur, t0=t0, b=b)

        # We'll also say that the timescale of the GP will be shared
        ell = pm.InverseGamma(
            "ell", testval=2.0, **xo.estimate_inverse_gamma_parameters(1.0, 5.0)
        )

        # Loop over the instruments
        parameters = dict()
        lc_models = dict()
        gp_preds = dict()
        gp_preds_with_mean = dict()
        for n, (name, (x, y, yerr, texp)) in enumerate(datasets.items()):

            # We define the per-instrument parameters in a submodel so that we
            # don't have to prefix the names manually
            with pm.Model(name=name, model=model):
                # The flux zero point
                mean = pm.Normal("mean", mu=0.0, sigma=10.0)

                # The limb darkening
                u = xo.distributions.QuadLimbDark("u")
                star = xo.LimbDarkLightCurve(u)

                # The radius ratio
                approx_depth = pm.Lognormal("approx_depth", mu=np.log(4e-3), sigma=10)
                ld = 1 - u[0] * ld_arg - u[1] * ld_arg ** 2
                ror = pm.Deterministic("ror", tt.sqrt(approx_depth / ld))

                # Noise parameters
                med_yerr = np.median(yerr)
                std = np.std(y)
                sigma = pm.InverseGamma(
                    "sigma",
                    testval=med_yerr,
                    **xo.estimate_inverse_gamma_parameters(med_yerr, 0.5 * std),
                )
                S_tot = pm.InverseGamma(
                    "S_tot",
                    testval=med_yerr,
                    **xo.estimate_inverse_gamma_parameters(
                        med_yerr ** 2, 0.25 * std ** 2
                    ),
                )

                # Keep track of the parameters for optimization
                parameters[name] = [mean, u, approx_depth]
                parameters[f"{name}_noise"] = [sigma, S_tot]

            # The light curve model
            def lc_model(mean, star, ror, texp, t):
                return mean + 1e3 * tt.sum(
                    star.get_light_curve(orbit=orbit, r=ror, t=t, texp=texp), axis=-1
                )

            lc_model = partial(lc_model, mean, star, ror, texp)
            lc_models[name] = lc_model

            # The Gaussian Process noise model
            kernel = xo.gp.terms.SHOTerm(S_tot=S_tot, w0=2 * np.pi / ell, Q=1.0 / 3)
            gp = xo.gp.GP(kernel, x, yerr ** 2 + sigma ** 2, mean=lc_model)
            gp.marginal(f"{name}_obs", observed=y)
            gp_preds[name] = gp.predict()
            gp_preds_with_mean[name] = gp.predict(predict_mean=True)

        # Optimize the model
        map_soln = model.test_point
        for name in datasets:
            map_soln = xo.optimize(map_soln, parameters[name])
        for name in datasets:
            map_soln = xo.optimize(map_soln, parameters[name] + [dur, b])
            map_soln = xo.optimize(map_soln, parameters[f"{name}_noise"])
        map_soln = xo.optimize(map_soln)

        # Do some sigma clipping
        num = dict((name, len(datasets[name][0])) for name in datasets)
        clipped = dict()
        masks = dict()
        for name in datasets:
            mdl = xo.eval_in_model(gp_preds_with_mean[name], map_soln)
            resid = datasets[name][1] - mdl
            sigma = np.sqrt(np.median((resid - np.median(resid)) ** 2))
            masks[name] = np.abs(resid - np.median(resid)) < 7 * sigma
            clipped[name] = num[name] - masks[name].sum()
            print(f"Sigma clipped {clipped[name]} {name} light curve points")
        if all(c < 10 for c in clipped.values()):
            break

        else:
            for name in datasets:
                datasets[name][0] = datasets[name][0][masks[name]]
                datasets[name][1] = datasets[name][1][masks[name]]
                datasets[name][2] = datasets[name][2][masks[name]]

In [ ]:
map_soln

In [ ]:
dt = np.linspace(-0.2, 0.2, 500)

with model:
    trends = xo.eval_in_model([gp_preds[k] for k in datasets], map_soln)
    phase_curves = xo.eval_in_model([lc_models[k](t0 + dt) for k in datasets], map_soln)

fig, axes = plt.subplots(2, sharex=True, sharey=True, figsize=(8, 6))

for n, name in enumerate(datasets):
    ax = axes[n]

    x, y = datasets[name][:2]

    period = map_soln["period"]
    folded = (x - map_soln["t0"] + 0.5 * period) % period - 0.5 * period
    m = np.abs(folded) < 0.2
    ax.plot(
        folded[m],
        (y - trends[n] - map_soln[f"{name}_mean"])[m],
        ".k",
        alpha=0.3,
        mec="none",
    )
    ax.plot(dt, phase_curves[n] - map_soln[f"{name}_mean"], f"C{n}", label=name)
    ax.annotate(
        name,
        xy=(1, 0),
        xycoords="axes fraction",
        va="bottom",
        ha="right",
        xytext=(-3, 3),
        textcoords="offset points",
        fontsize=14,
    )

axes[-1].set_xlim(-0.15, 0.15)
axes[-1].set_xlabel("time since transit [days]")
for ax in axes:
    ax.set_ylabel("relative flux [ppt]")

In [ ]:
np.random.seed(11)
with model:
    trace = xo.sample(
        tune=3500, draws=3000, start=map_soln, chains=4, initial_accept=0.5,
    )

In [ ]:
pm.summary(trace)

In [ ]:
plt.hist(trace["Kepler_ror"], 30, density=True, histtype="step")
plt.hist(trace["TESS_ror"], 30, density=True, histtype="step");

In [ ]:
import corner

fig = corner.corner(trace["Kepler_u"], color="C0")
corner.corner(trace["TESS_u"], color="C1", fig=fig, labels=["$u_1$", "$u_2$"]);